In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
from functools import reduce

%matplotlib inline

### Solo Exploration and Presentation:
#### 1. Choose and download another data set from the UN data to explore.
[http://data.un.org/Explorer.aspx](http://data.un.org/Explorer.aspx)   
You may want to combine your new dataset with one or both of the datasets that you already worked with. Prepare a short (< 5 minute) presentation of your findings. Report any interesting correlations or trends that you find. 
#### 2. If time allows, check out the plotly library to add additional interativity to your plots. 
[https://plotly.com/python/plotly-express/](https://plotly.com/python/plotly-express/)

#### Primary Education Completion Rates

In [2]:
endpoint1 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/Ief9e0d38__Sc5211ad2_1644293535310441_tbl/FeatureServer/0/query?where=income_wealth_quantile%20%3D%20\'_T\'%20AND%20urbanization%20%3D%20\'_T\'%20AND%20time_period%20%3E%3D%202004%20AND%20time_period%20%3C%3D%202019&outFields=time_period,sex,sex_desc,education_lev_desc,urbanization_desc,income_wealth_quantile_desc,obs_value,unit_measure_desc,indicator_desc,ref_area_desc&returnGeometry=false&outSR=&f=json'

endpoint2 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/Ief9e0d38__Sc5211ad2_1644293535310441_tbl/FeatureServer/0/query?where=income_wealth_quantile%20%3D%20\'_T\'%20AND%20urbanization%20%3D%20\'_T\'%20AND%20time_period%20%3E%3D%202004%20AND%20time_period%20%3C%3D%202019&outFields=time_period,sex,sex_desc,education_lev_desc,urbanization_desc,income_wealth_quantile_desc,obs_value,unit_measure_desc,indicator_desc,ref_area_desc&returnGeometry=false&resultOffset=1000&outSR=&f=json'

In [3]:
pe_res1 = requests.get(endpoint1).json()
pe_res2 = requests.get(endpoint2).json()

In [4]:
pe_df1 = pd.DataFrame(pd.DataFrame(pe_res1['features'])['attributes'].values.tolist())
pe_df2 = pd.DataFrame(pd.DataFrame(pe_res2['features'])['attributes'].values.tolist())

primary_ed = pd.concat([pe_df1, pe_df2])
primary_ed = primary_ed[['ref_area_desc', 'time_period', 'sex', 'obs_value']]
primary_ed.columns = ['Country', 'Year', 'Sex', 'Primary_ed_completion_percent']
primary_ed = primary_ed.sort_values(['Country', 'Year', 'Sex'])
primary_ed

,Country,Year,Sex,Primary_ed_completion_percent
625,Afghanistan,2011,F,26.63788
228,Afghanistan,2011,M,53.74893
26,Afghanistan,2011,_T,40.72569
577,Afghanistan,2015,F,40.35416
229,Afghanistan,2015,M,67.30569
...,...,...,...,...
834,Zimbabwe,2015,M,87.02327
623,Zimbabwe,2015,_T,88.21310
227,Zimbabwe,2019,F,91.62107
835,Zimbabwe,2019,M,86.35518


#### Secondary Education Completion

In [5]:
se1 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/I66c21045__Sa7501ddf_16442936354058118_tbl/FeatureServer/0/query?where=urbanization%20%3D%20\'_T\'%20AND%20income_wealth_quantile%20%3D%20\'_T\'%20AND%20time_period%20%3E%3D%202004%20AND%20time_period%20%3C%3D%202019&outFields=indicator_desc,ref_area_desc,time_period,sex,education_lev_desc,obs_value,unit_measure_desc&returnGeometry=false&outSR=&f=json'
se2 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/I66c21045__Sa7501ddf_16442936354058118_tbl/FeatureServer/0/query?where=urbanization%20%3D%20\'_T\'%20AND%20income_wealth_quantile%20%3D%20\'_T\'%20AND%20time_period%20%3E%3D%202004%20AND%20time_period%20%3C%3D%202019&outFields=indicator_desc,ref_area_desc,time_period,sex,education_lev_desc,obs_value,unit_measure_desc&returnGeometry=false&resultOffset=1000&outSR=&f=json'
se3 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/I66c21045__Sa7501ddf_16442936354058118_tbl/FeatureServer/0/query?where=urbanization%20%3D%20\'_T\'%20AND%20income_wealth_quantile%20%3D%20\'_T\'%20AND%20time_period%20%3E%3D%202004%20AND%20time_period%20%3C%3D%202019&outFields=indicator_desc,ref_area_desc,time_period,sex,education_lev_desc,obs_value,unit_measure_desc&returnGeometry=false&resultOffset=2000&outSR=&f=json'

se_res1 = requests.get(se1).json()
se_res2 = requests.get(se2).json()
se_res3 = requests.get(se3).json()

In [6]:
se_df1 = pd.DataFrame(pd.DataFrame(se_res1['features'])['attributes'].values.tolist())
se_df2 = pd.DataFrame(pd.DataFrame(se_res2['features'])['attributes'].values.tolist())
se_df3 = pd.DataFrame(pd.DataFrame(se_res3['features'])['attributes'].values.tolist())

secondary_ed = pd.concat([se_df1, se_df2, se_df3])
secondary_ed = secondary_ed[['ref_area_desc', 'time_period', 'sex', 'obs_value']]
secondary_ed.columns = ['Country', 'Year', 'Sex', 'Secondary_ed_completion_percent']
secondary_ed = secondary_ed.sort_values(['Country', 'Year', 'Sex'])

secondary_ed

,Country,Year,Sex,Secondary_ed_completion_percent
768,Afghanistan,2011,F,11.50881
512,Afghanistan,2011,M,34.65255
19,Afghanistan,2011,_T,23.37700
769,Afghanistan,2015,F,25.17210
464,Afghanistan,2015,M,48.71595
...,...,...,...,...
164,Zimbabwe,2015,M,72.22249
766,Zimbabwe,2015,_T,72.62408
511,Zimbabwe,2019,F,54.36128
165,Zimbabwe,2019,M,53.50781


#### Maternal Death Rate per 100,000 live births

In [7]:
mdr1 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/I75250979__S75250979_16442942938748052_tbl/FeatureServer/0/query?where=1%3D1&outFields=indicator_desc,ref_area_desc,time_period,obs_value,unit_measure_desc&returnGeometry=false&outSR=&f=json'
mdr2 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/I75250979__S75250979_16442942938748052_tbl/FeatureServer/0/query?where=1%3D1&outFields=indicator_desc,ref_area_desc,time_period,obs_value,unit_measure_desc&returnGeometry=false&outSR=&resultOffset=1000&f=json'
mdr3 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/I75250979__S75250979_16442942938748052_tbl/FeatureServer/0/query?where=1%3D1&outFields=indicator_desc,ref_area_desc,time_period,obs_value,unit_measure_desc&returnGeometry=false&outSR=&resultOffset=2000&f=json'
mdr4 = 'https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/I75250979__S75250979_16442942938748052_tbl/FeatureServer/0/query?where=1%3D1&outFields=indicator_desc,ref_area_desc,time_period,obs_value,unit_measure_desc&returnGeometry=false&outSR=&resultOffset=3000&f=json'

In [8]:
mdr_res1 = requests.get(mdr1).json()
mdr_res2 = requests.get(mdr2).json()
mdr_res3 = requests.get(mdr3).json()
mdr_res4 = requests.get(mdr4).json()

In [9]:
mdr_df1 = pd.DataFrame(pd.DataFrame(mdr_res1['features'])['attributes'].values.tolist())
mdr_df2 = pd.DataFrame(pd.DataFrame(mdr_res2['features'])['attributes'].values.tolist())
mdr_df3 = pd.DataFrame(pd.DataFrame(mdr_res3['features'])['attributes'].values.tolist())
mdr_df4 = pd.DataFrame(pd.DataFrame(mdr_res4['features'])['attributes'].values.tolist())

mdr = pd.concat([mdr_df1, mdr_df2, mdr_df3, mdr_df4])
mdr =(
    mdr.drop(columns = ['indicator_desc', 'unit_measure_desc'])
    .rename(columns = {'ref_area_desc' : 'Country',
                      'time_period' : 'Year',
                      'obs_value' : 'Maternal_Death_Rate'})
)

mdr = mdr.sort_values(['Country', 'Year'])

mdr

,Country,Year,Maternal_Death_Rate
184,Afghanistan,2000,1450
185,Afghanistan,2001,1390
186,Afghanistan,2002,1300
187,Afghanistan,2003,1240
188,Afghanistan,2004,1180
...,...,...,...
465,Zimbabwe,2013,509
466,Zimbabwe,2014,494
467,Zimbabwe,2015,480
468,Zimbabwe,2016,468


#### Merging the two education tables

In [10]:
education = pd.merge(primary_ed, secondary_ed, how='outer', on=['Country', 'Year', 'Sex'])

#Looking at rows that don't have primary education rates listed
(
    education.loc[(education['Primary_ed_completion_percent']
                   .isna())]
)

,Country,Year,Sex,Primary_ed_completion_percent,Secondary_ed_completion_percent
1836,Australia,2010,F,NaN,99.53
1837,Australia,2010,M,NaN,98.82
1838,Australia,2010,_T,NaN,99.15
1839,Austria,2005,F,NaN,98.59
1840,Austria,2005,M,NaN,98.73
...,...,...,...,...,...
2188,United Kingdom of Great Britain and Northern I...,2013,M,NaN,100.00
2189,United Kingdom of Great Britain and Northern I...,2013,_T,NaN,100.00
2190,United Kingdom of Great Britain and Northern I...,2014,F,NaN,100.00
2191,United Kingdom of Great Britain and Northern I...,2014,M,NaN,100.00


#### Bringing in the other data sets

In [11]:
gdp_df = pd.read_csv('../data/gdp_per_capita.csv', nrows=6868)
#This prevents bringing in the rows containing footnotes at the bottom of the file

continents = pd.read_csv('../data/continents.csv')

life_expectancy = pd.read_csv('../data/life_expectancy.csv', header=2)

In [12]:
gdp_df = gdp_df.drop(columns = 'Value Footnotes')
gdp_df.columns = ['Country', 'Year', 'GDP_Per_Capita']
gdp_df['Country'] = gdp_df['Country'].str.replace('The', '')
gdp_df['Country'] = gdp_df['Country'].str.replace('Democratic Republic of the', 'Dem. Rep.')

continents = continents.loc[~continents['Country'].str.contains('The')]

life_expectancy = life_expectancy.drop(columns = ['Country Code', 'Indicator Name', 
                                                  'Indicator Code', '2021', 'Unnamed: 66'])
life_expectancy = life_expectancy.melt(id_vars=['Country Name']).dropna()
life_expectancy.columns = ['Country', 'Year', 'Life_Expectancy']

#### Looking at mismatched country names

In [32]:
pattern = '|'.join([', The', ', Rep.', ', RB', ', Arab Rep.', ', Islamic Rep.', 
                    ', Fed. Sts.', ' \(Plurinational State of\)', ' \(Bolivarian Republic of\)',
                   ' \(Islamic Republic of\)', ' \(Dutch part\)'])

country_map = {
    'Democratic Republic of the Congo' : 'Dem. Rep. Congo',
    'Congo, Dem. Rep.' : 'Dem. Rep. Congo',
    'Burma (Myanmar)' : 'Myanmar',
    'Kyrgyzstan' : 'Kyrgyz Republic'
}

def country_name_counts(df_name, col):
    """Pulling out country names and value counts from each dataframe"""
    df_name['Country'] = df_name['Country'].str.replace(pattern, '', regex=True)
    df_name['Country'] = df_name['Country'].str.strip()
    df_name['Country'] = df_name['Country'].replace(country_map)
    df_name = df_name.drop_duplicates()
    co_df = (
        df_name['Country']
        .value_counts()
        .to_frame()
        .reset_index()
        .rename(columns = {'index' : 'Country',
                           'Country' : f'Count_{col}'})
    )
    return co_df

In [33]:
gdp_co = country_name_counts(gdp_df, "gdp")
le_co = country_name_counts(life_expectancy, "le")
continent_co = country_name_counts(continents, "continent")
ed_co = country_name_counts(education, "ed")
mdr_co = country_name_counts(mdr, "mdr")

In [34]:
country_dfs = [gdp_co, le_co, continent_co, ed_co, mdr_co]

all_countries = (
    reduce(lambda  left,right: pd.merge(left,right,on=['Country'],
                                        how='outer'), country_dfs)
)

all_countries = all_countries.loc[~all_countries['Country'].str.contains('Europe') &
                                  ~all_countries['Country'].str.contains('Asia') &
                                  ~all_countries['Country'].str.contains('Latin America') &
                                  ~all_countries['Country'].str.contains('\(') &
                                  ~all_countries['Country'].str.contains('countries') &
                                  ~all_countries['Country'].str.contains('Central') & 
                                  ~all_countries['Country'].str.contains('Western') &
                                  ~all_countries['Country'].str.contains('Sub') &
                                  ~all_countries['Country'].str.contains('World') &
                                  ~all_countries['Country'].str.contains('Korea') & 
                                  ~all_countries['Country'].str.contains('income')]

mismatch = (
    all_countries.loc[all_countries.isna()
                      .any(axis=1)]
    .sort_values('Country')
    .reset_index(drop=True)
)

mismatch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          128 non-null    object 
 1   Count_gdp        72 non-null     float64
 2   Count_le         85 non-null     float64
 3   Count_continent  73 non-null     float64
 4   Count_ed         20 non-null     float64
 5   Count_mdr        57 non-null     float64
dtypes: float64(5), object(1)
memory usage: 6.1+ KB


In [35]:
mismatch.head(60)

,Country,Count_gdp,Count_le,Count_continent,Count_ed,Count_mdr
0,Africa Eastern and Southern,NaN,61.0,NaN,NaN,NaN
1,Andorra,NaN,NaN,1.0,NaN,NaN
2,Antigua and Barbuda,30.0,61.0,1.0,NaN,18.0
3,Aruba,28.0,61.0,1.0,NaN,NaN
4,Australia and New Zealand,NaN,NaN,NaN,NaN,18.0
5,Bahamas,30.0,61.0,1.0,NaN,18.0
6,Bahrain,30.0,61.0,1.0,NaN,18.0
7,Bermuda,30.0,25.0,1.0,NaN,NaN
8,Brunei,30.0,NaN,1.0,NaN,NaN
9,Brunei Darussalam,NaN,61.0,NaN,NaN,18.0


In [36]:
all_countries.loc[all_countries['Country'].str.contains('Antigua')]

,Country,Count_gdp,Count_le,Count_continent,Count_ed,Count_mdr
139,Antigua and Barbuda,30.0,61.0,1.0,NaN,18.0


In [37]:
life_expectancy['Year'] = life_expectancy['Year'].astype(int)
gdp_le = (
    pd.merge(gdp_df, life_expectancy.loc[life_expectancy['Year'] > 1989], how='outer', on=['Country', 'Year'])
    .dropna(subset=['GDP_Per_Capita', 'Life_Expectancy'], how='all')
)
gdp_le.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8184 entries, 0 to 8183
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          8184 non-null   object 
 1   Year             8184 non-null   int64  
 2   GDP_Per_Capita   6868 non-null   float64
 3   Life_Expectancy  7669 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 319.7+ KB


In [38]:
gdp_le = pd.merge(gdp_le, continents, how='left', on='Country')
gdp_le.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8215 entries, 0 to 8214
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          8215 non-null   object 
 1   Year             8215 non-null   int64  
 2   GDP_Per_Capita   6898 non-null   float64
 3   Life_Expectancy  7700 non-null   float64
 4   Continent        6083 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 385.1+ KB


In [39]:
continents.loc[continents['Country'].duplicated(keep=False)]

,Continent,Country
30,Asia,Myanmar
132,Asia,Myanmar
